In [1]:
import clip
import argparse
import numpy as np

from clip.trainer import CLIPTrainer
from loaders import CxrDataLoader
from constants import VIT_TYPE

In [2]:
args = argparse.Namespace(
    dataset_name='iu-xray',
    batch_size=16,
    num_workers=0,
    max_seq_length=248
)

In [4]:
model, preprocess = clip.load("ViT-B/32", load_from_clip=True, extended_context=False)

input_resolution = model.visual.input_resolution
context_length = model.context_length
extended_context_length = model.extended_context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Original context length:", context_length)
print("Extended context length:", extended_context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,685,391
Input resolution: 224
Original context length: 77
Extended context length: 248
Vocab size: 49408


In [4]:
dataloaders = {
    'train': CxrDataLoader(args, split='train', transform=preprocess),
    'val': CxrDataLoader(args, split='val', transform=preprocess),
    'test': CxrDataLoader(args, split='test', transform=preprocess)
}

In [5]:
trainer = CLIPTrainer(
    model,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    test_loader=dataloaders['test'],
    log_interval=20
)

Computing pos weights: 100%|██████████| 169/169 [01:44<00:00,  1.62it/s]
2025-03-30 16:36:10,749 - INFO - Total parameters: 151,899,919
2025-03-30 16:36:10,749 - INFO - Trainable parameters: 151,772,943 (99.92%)


In [6]:
trainer.train(epochs=5)

2025-03-30 16:36:10,765 - INFO - Starting training
2025-03-30 16:36:10,766 - INFO - Starting epoch 1/5
100%|██████████| 169/169 [07:52<00:00,  2.79s/it]
2025-03-30 16:44:02,819 - INFO - Epoch 1 completed in 472.05s. Average loss: 3.3703
2025-03-30 16:44:02,820 - INFO - Running evaluation on val set
Val Evaluation: 100%|██████████| 37/37 [00:49<00:00,  1.33s/it]
2025-03-30 16:44:52,018 - INFO - Val metrics: {
  "val_loss": 2.8194871883134582,
  "val_best_threshold": 0.5,
  "val_mean_f1": 0.12194698841703863,
  "val_mean_precision": 0.1581813205342367,
  "val_mean_recall": 0.3411072144501444,
  "val_concept_auc_avg": 0.6986746954026913
}
2025-03-30 16:44:53,640 - INFO - Saved best model to ./output/clip_cxr_model.pt
2025-03-30 16:44:53,640 - INFO - Starting epoch 2/5
100%|██████████| 169/169 [07:47<00:00,  2.77s/it]
2025-03-30 16:52:41,457 - INFO - Epoch 2 completed in 467.82s. Average loss: 3.0702
2025-03-30 16:52:41,458 - INFO - Running evaluation on val set
Val Evaluation: 100%|██████